In [1]:
#install packages
import pandas as pd
import numpy as np
import json
import requests
from io import StringIO
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from math import ceil
import datetime
import calendar
from sqlalchemy import create_engine

In [2]:
# Print the configuration
connection_string = "connection_string"

# Set up connection string for Azure storage and container name
connection_string_azure_storage = connection_string
container_azure = 'nypdarrests'

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string_azure_storage)

# Get the container client
container_client = blob_service_client.get_container_client(container_azure)


nypdarrests_df = pd.DataFrame()

# List all blobs in the specified container
blob_list = container_client.list_blobs()
for blob in blob_list:
    print(blob.name)
    blob_client = container_client.get_blob_client(blob=blob.name)
    blob_data = blob_client.download_blob()
    blob_content = blob_data.readall().decode('utf-8')
    df = pd.read_csv(StringIO(blob_content))
    # Display the head of the DataFrame
    print(df.head())
    # If I have only one csv, I am doing the following instructions
    nypdarrests_df = df.copy()

nypdarrests.csv
   ARREST_KEY ARREST_DATE  PD_CD                         PD_DESC  KY_CD  \
0   261265483  01/03/2023  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
1   261271301  01/03/2023  105.0               STRANGULATION 1ST  106.0   
2   261336449  01/04/2023  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
3   261328047  01/04/2023  105.0               STRANGULATION 1ST  106.0   
4   261417496  01/05/2023  244.0   BURGLARY,UNCLASSIFIED,UNKNOWN  107.0   

        OFNS_DESC    LAW_CODE LAW_CAT_CD ARREST_BORO  ARREST_PRECINCT  \
0         ROBBERY  PL 1600500          F           B               49   
1  FELONY ASSAULT  PL 1211200          F           S              120   
2         ROBBERY  PL 1601001          F           K               61   
3  FELONY ASSAULT  PL 1211200          F           Q              114   
4        BURGLARY  PL 1402000          F           B               44   

   JURISDICTION_CODE AGE_GROUP PERP_SEX PERP_RACE  X_COORD_CD  Y_COORD_CD  \
0                

In [3]:
# Check for null values in the entire DataFrame
null_counts = nypdarrests_df.isnull().sum()

# Display the null counts
print("Null counts in the entire DataFrame:")
print(null_counts)


Null counts in the entire DataFrame:
ARREST_KEY                     0
ARREST_DATE                    0
PD_CD                          2
PD_DESC                        0
KY_CD                         17
OFNS_DESC                      0
LAW_CODE                       0
LAW_CAT_CD                  1599
ARREST_BORO                    0
ARREST_PRECINCT                0
JURISDICTION_CODE              0
AGE_GROUP                      0
PERP_SEX                       0
PERP_RACE                      0
X_COORD_CD                     0
Y_COORD_CD                     0
Latitude                       0
Longitude                      0
New Georeferenced Column       0
dtype: int64


In [4]:
# Drop rows with null values from the DataFrame
nypdarrests_df.dropna(inplace=True)

# Confirm that null values have been dropped
print("Null counts after dropping rows with null values:")
print(nypdarrests_df.isnull().sum())


Null counts after dropping rows with null values:
ARREST_KEY                  0
ARREST_DATE                 0
PD_CD                       0
PD_DESC                     0
KY_CD                       0
OFNS_DESC                   0
LAW_CODE                    0
LAW_CAT_CD                  0
ARREST_BORO                 0
ARREST_PRECINCT             0
JURISDICTION_CODE           0
AGE_GROUP                   0
PERP_SEX                    0
PERP_RACE                   0
X_COORD_CD                  0
Y_COORD_CD                  0
Latitude                    0
Longitude                   0
New Georeferenced Column    0
dtype: int64


In [5]:
nypdarrests_df.head()
min_date = nypdarrests_df['ARREST_DATE'].min()
max_date = nypdarrests_df['ARREST_DATE'].max()

print(min_date)
print(max_date)

01/01/2023
12/31/2023


In [6]:
# Load your lookup DataFrame, assuming lookup_df is already defined and loaded
dim_perp_df = nypdarrests_df[['ARREST_KEY', 'AGE_GROUP', 'PERP_SEX', 'PERP_RACE']]

# Define a rename mapping to the desired column names
rename_mapping = {
    'ARREST_KEY': 'arrest_id',
    'AGE_GROUP': 'age_group',
    'PERP_SEX': 'perp_sex',
    'PERP_RACE': 'perp_race',
}

# Rename the columns using the mapping
dim_perp_df.rename(columns=rename_mapping, inplace=True)

# Drop rows with missing values
dim_perp_df.dropna(inplace=True)

# Assign dim_perp_df to dim_perp
dim_perp = dim_perp_df

# Now, let's display the first few rows to confirm the changes
print(dim_perp.head())


   arrest_id age_group perp_sex perp_race
0  261265483     18-24        M     BLACK
1  261271301     25-44        M     WHITE
2  261336449       <18        M     BLACK
3  261328047     18-24        M     BLACK
4  261417496     25-44        F     BLACK


C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\3159256302.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_perp_df.rename(columns=rename_mapping, inplace=True)
C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\3159256302.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_perp_df.dropna(inplace=True)


In [7]:
# Load your lookup DataFrame, assuming lookup_df is already defined and loaded
dim_location_df = nypdarrests_df[['New Georeferenced Column', 'ARREST_BORO', 'ARREST_PRECINCT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude']]

# Define a rename mapping to the desired column names
rename_mapping = {
    'New Georeferenced Column': 'location_id',
    'ARREST_BORO': 'arrest_boro',
    'ARREST_PRECINCT': 'arrest_precinct',
    'X_COORD_CD': 'x_coord_cd',
    'Y_COORD_CD': 'y_coord_cd',
    'Latitude': 'latitude',
    'Longitude': 'longitude'  
}

# Rename the columns using the mapping
dim_location_df.rename(columns=rename_mapping, inplace=True)

# Drop rows with missing values
dim_location_df.dropna(inplace=True)

# Assign dim_location_df to dim_location
dim_location = dim_location_df

# Now, let's display the first few rows to confirm the changes
print(dim_location.head())


                    location_id arrest_boro  arrest_precinct  x_coord_cd  \
0  POINT (-73.843908 40.855793)           B               49     1027430   
1  POINT (-74.077263 40.644996)           S              120      962808   
2  POINT (-73.960866 40.594054)           K               61      995118   
3  POINT (-73.915361 40.769552)           Q              114     1007694   
4   POINT (-73.91717 40.824135)           B               44     1007174   

   y_coord_cd   latitude  longitude  
0      251104  40.855793 -73.843908  
1      174275  40.644996 -74.077263  
2      155708  40.594054 -73.960866  
3      219656  40.769552 -73.915361  
4      239542  40.824135 -73.917170  


C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\4109082177.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_location_df.rename(columns=rename_mapping, inplace=True)
C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\4109082177.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_location_df.dropna(inplace=True)


In [8]:
# Load your lookup DataFrame, assuming lookup_df is already defined and loaded
dim_date_df = nypdarrests_df[['ARREST_DATE']]

# Define a rename mapping to the desired column names
rename_mapping = {
    'ARREST_DATE': 'date_id',
    
}

# Rename the columns using the mapping
dim_date_df.rename(columns=rename_mapping, inplace=True)

# Drop rows with missing values
dim_date_df.dropna(inplace=True)

# Assign dim_location_df to dim_location
dim_date = dim_date_df

# Now, let's display the first few rows to confirm the changes
print(dim_date.head())


      date_id
0  01/03/2023
1  01/03/2023
2  01/04/2023
3  01/04/2023
4  01/05/2023


C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\2520331667.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date_df.rename(columns=rename_mapping, inplace=True)
C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\2520331667.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_date_df.dropna(inplace=True)


In [9]:
import calendar

def week_of_month(dt):
    year = dt.year
    month = dt.month
    day = dt.day
    cal = calendar.monthcalendar(year, month)
    for i, week in enumerate(cal):
        if day in week:
            return i + 1

start_date = pd.to_datetime('2023-01-03')
end_date = pd.to_datetime('2023-10-18')

# Create a DataFrame for the date dimension
date_dimension = pd.DataFrame({'date': pd.date_range(start_date, end_date, freq='H')})

# Extract attributes
date_dimension['date_id'] = date_dimension['date'].dt.strftime('%Y-%m-%d')  # Adjusted for correct date format
date_dimension['year'] = date_dimension['date'].dt.year
date_dimension['quarter'] = date_dimension['date'].dt.quarter
date_dimension['month'] = date_dimension['date'].dt.month
date_dimension['day'] = date_dimension['date'].dt.day
date_dimension['weekday'] = date_dimension['date'].dt.strftime('%A')

# Set the new order for columns as per requirement
new_order = ['date_id']
date_dimension = date_dimension[new_order]


print(date_dimension.head())


      date_id
0  2023-01-03
1  2023-01-03
2  2023-01-03
3  2023-01-03
4  2023-01-03


In [10]:
# Load your lookup DataFrame, assuming lookup_df is already defined and loaded
dim_offensetype_df = nypdarrests_df[['OFNS_DESC', 'JURISDICTION_CODE', 'LAW_CODE']]

# Define a rename mapping to the desired column names
rename_mapping = {
    'OFNS_DESC': 'offenseType_id',
    'JURISDICTION_CODE': 'jurisdiction_code',
    'LAW_CODE': 'law_code',   
}

# Rename the columns using the mapping
dim_offensetype_df.rename(columns=rename_mapping, inplace=True)

# Drop rows with missing values
dim_offensetype_df.dropna(inplace=True)

# Assign dim_offensetype_df to dim_offenseType
dim_offenseType = dim_offensetype_df

# Now, let's display the first few rows to confirm the changes
print(dim_offenseType.head())


   offenseType_id  jurisdiction_code    law_code
0         ROBBERY                  0  PL 1600500
1  FELONY ASSAULT                  0  PL 1211200
2         ROBBERY                  0  PL 1601001
3  FELONY ASSAULT                  0  PL 1211200
4        BURGLARY                  0  PL 1402000


C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\2255089306.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_offensetype_df.rename(columns=rename_mapping, inplace=True)
C:\Users\raisa\AppData\Local\Temp\ipykernel_21568\2255089306.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_offensetype_df.dropna(inplace=True)


In [11]:
import pandas as pd

# Assuming nypdarrests_df is your DataFrame

# Calculate mode for PERP_SEX and PERP_RACE
mode_perpsex = nypdarrests_df['PERP_SEX'].mode()[0]
mode_perprace = nypdarrests_df['PERP_RACE'].mode()[0]

# Add new columns for mode of PERP_SEX and PERP_RACE
nypdarrests_df['mode_perpsex'] = mode_perpsex
nypdarrests_df['mode_perprace'] = mode_perprace

# Print the updated DataFrame
print(nypdarrests_df.head())


   ARREST_KEY ARREST_DATE  PD_CD                         PD_DESC  KY_CD  \
0   261265483  01/03/2023  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
1   261271301  01/03/2023  105.0               STRANGULATION 1ST  106.0   
2   261336449  01/04/2023  397.0  ROBBERY,OPEN AREA UNCLASSIFIED  105.0   
3   261328047  01/04/2023  105.0               STRANGULATION 1ST  106.0   
4   261417496  01/05/2023  244.0   BURGLARY,UNCLASSIFIED,UNKNOWN  107.0   

        OFNS_DESC    LAW_CODE LAW_CAT_CD ARREST_BORO  ARREST_PRECINCT  ...  \
0         ROBBERY  PL 1600500          F           B               49  ...   
1  FELONY ASSAULT  PL 1211200          F           S              120  ...   
2         ROBBERY  PL 1601001          F           K               61  ...   
3  FELONY ASSAULT  PL 1211200          F           Q              114  ...   
4        BURGLARY  PL 1402000          F           B               44  ...   

   AGE_GROUP PERP_SEX PERP_RACE X_COORD_CD  Y_COORD_CD   Latitude  Longitude  \


In [12]:
# Create FactID column
nypdarrests_df['facts_id'] = range(1, len(nypdarrests_df) + 1)
nypdarrests_df['ARREST_DATE'] = date_dimension

# Rename columns
new_column_names = {
    'facts_id': 'facts_id',
    'ARREST_KEY': 'arrest_id',
    'OFNS_DESC': 'offenseType_id',
    'mode_perpsex': 'mode_perpsex',
    'mode_perprace': 'mode_perprace',
    'New Georeferenced Column': 'location_id',
    'ARREST_DATE': 'date_id',
    
    
    
}

nypdarrests_df = nypdarrests_df.rename(columns=new_column_names)

# Define the columns
required_columns = ['facts_id', 'arrest_id', 'offenseType_id', 'mode_perpsex','mode_perprace','location_id','date_id']

nypdarrests = nypdarrests_df[required_columns]

print(nypdarrests.columns) 

# Display the DataFrame
nypdarrests.head()

Index(['facts_id', 'arrest_id', 'offenseType_id', 'mode_perpsex',
       'mode_perprace', 'location_id', 'date_id'],
      dtype='object')


,facts_id,arrest_id,offenseType_id,mode_perpsex,mode_perprace,location_id,date_id
0,1,261265483,ROBBERY,M,BLACK,POINT (-73.843908 40.855793),2023-01-03
1,2,261271301,FELONY ASSAULT,M,BLACK,POINT (-74.077263 40.644996),2023-01-03
2,3,261336449,ROBBERY,M,BLACK,POINT (-73.960866 40.594054),2023-01-03
3,4,261328047,FELONY ASSAULT,M,BLACK,POINT (-73.915361 40.769552),2023-01-03
4,5,261417496,BURGLARY,M,BLACK,POINT (-73.91717 40.824135),2023-01-03


In [13]:
# Assuming dim_perp, dim_location, dim_offenseType are your dimension DataFrames

# Convert dim_perp to CSV
try:
    dim_perp.to_csv('dim_perp.csv', index=False)
    print("dim_perp.csv created successfully.")
except Exception as e:
    print("Error occurred while creating dim_perp.csv:", str(e))

# Convert dim_location to CSV
try:
    dim_location.to_csv('dim_location.csv', index=False)
    print("dim_location.csv created successfully.")
except Exception as e:
    print("Error occurred while creating dim_location.csv:", str(e))

# Convert dim_offenseType to CSV
try:
    dim_offenseType.to_csv('dim_offenseType.csv', index=False)
    print("dim_offenseType.csv created successfully.")
except Exception as e:
    print("Error occurred while creating dim_offenseType.csv:", str(e))


dim_perp.csv created successfully.
dim_location.csv created successfully.
dim_offenseType.csv created successfully.


In [14]:
# Assuming nypdarrests_df is your DataFrame for 'nypdarrests' dimension
# and date_df is your DataFrame for 'date' dimension

# Convert nypdarrests_df to CSV
try:
    nypdarrests_df.to_csv('nypdarrests.csv', index=False)
    print("nypdarrests.csv created successfully.")
except Exception as e:
    print("Error occurred while creating nypdarrests.csv:", str(e))

# Convert date_df to CSV
try:
    date_dimension.to_csv('date.csv', index=False)
    print("date.csv created successfully.")
except Exception as e:
    print("Error occurred while creating date.csv:", str(e))


nypdarrests.csv created successfully.
date.csv created successfully.
